<div align="left">
<img src="../images/unige.png" width="10%" height="10%" title="University of Genoa" alt="University of Genoa" >
</div>


# Reasearch Track II Assignment 2

---

**Author's Name: Omotoye Shamsudeen Adekoya** <br>
**Student ID: 5066348**

---

## NoteBook Description

This Notebook has two functions; it serves as a user interface for the control of non-holonomic 2 Wheeled robot in a 3D environment and it also show a graphical feedback of the robot state and behaviour. 

<div align="center">
    <h1> Robot Controller </h1>
<img src="../images/rviz-robotmodel.png" width="35%" height="35%" title="Two Wheel Non-Holonomic Robot" alt="Two Wheeled Non-Holonomic Robot Model" >
</div>

### Importing all the necessary libraries
List of Libraries implemented for the control and visualization operation

---
|Library|Use|
|:---|:---|
|**rospy**|Used for the Initialization of the NoteBook as a ROS Node and also for implementing all the ros communication mechanism (**Subscriber, Publisher, Service, \*Parameter Server**) to communicate with all the other nodes in order to control and receive information from the Mobile Robot|
|**IpyWidgets**| Used for the creation of the buttons and slider for sending commands to the ros nodes |
|**matplotlib**| Used for creating interactive Visualization of the data returned from the robot about the **robot state**|
|**tf**| Used for interpreting the Odometry data by converting the orientation data from quaternion to euler's angle |
|**all_buttons**| It contains all the button object created for the Notebook controller |



In [1]:
import ipywidgets as widgets
from ipywidgets import Button, Layout, HBox, VBox, GridBox, Layout, ButtonStyle 

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Button as Btn
from matplotlib.lines import Line2D 
from matplotlib.patches import Arrow, Circle

import rospy
import tf
from nav_msgs.msg import Odometry
from rt2_assignment2.srv import Command, RandomPosition
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry 
from tf.transformations import euler_from_quaternion

from math import *
import time

from all_buttons import *

%matplotlib widget
plt.style.use('ggplot')

### Figure Classes

The class was created, **_Visualiser(), AnalysisPlots(), VelocityAnalysis()_**
 
* __Visualiser()__: _This class handles the figure that displays the position and orientation of the Robot and also information about the goal position and the distance of the robot to the goal_
* __AnalysisPlots()__: _This class handles the figure that displays the **BarPlot** of the reached and canceled target as well as the **Histogram** showing the frequency of time the robot takes to reach a goal target._
* __VelocityAnalysis()__: _This class handles the figure that displays the **Line Plots** for visualising the velocity send through the **cmd_vel** topic and the **actual velocity** of the robot_

In [2]:
class Visualiser:
    """This class is for the creation of the Figure which contains the graphical 
    and textual representation of the robot state (position and orientation) 
    and the goal position
    """
    def __init__(self):
        # Creating the figure for adding Axes
        self.fig = plt.figure(figsize=(8,6))
        
        ## Adding an axes to the created figure
        self.ax = self.fig.add_axes([0.45,0.17,0.52,0.70]) # here is where the sectioning of the axis of the figure is done 
        self.ax.set_title('Robot Position and Goal Visualizer')
        self.ax.set_xlabel('X Coordinate')
        self.ax.set_ylabel('Y Coordinate')
        
        # Creating and adding a line to the axes created
        self.line = Line2D([],[], color='g', label='Motion Path', marker='.',
     markerfacecolor='blue')
        self.ax.add_line(self.line)
        
        ## Creating and adding an arrow artist to the axes
        # this arrow help to visulize the orientation of the robot 
        self.arrow = Arrow(0, 0, 0, 0)
        self.ax.add_patch(self.arrow)
        
        ## Creating and adding a circle artist to the axes
        # this circle helps to visualize the position of the robot 
        self.circle = Circle((0, 0), radius=0, label='Robot Pose')
        self.ax.add_patch(self.circle)
        
        # Creating and adding a circle artist representing the goal point
        self.target = Circle((0, 0), radius=0,  color='r',label='Goal Point')
        self.ax.add_patch(self.target)
        
        # Creating an axes for displaying all the textual information
        # this axes snows the value of the robot position, orientation, distance to goal etc.....
        self.info_ax = self.fig.add_axes([0.05,0.05, 0.30,0.9])
        self.info_ax.grid(False)
        self.info_ax.set_facecolor('#faebd7')
        self.info_ax.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
        
        ## Creating an axes for displaying a button
        # this button is used for clear the path that has been plotted on the axes for displaying the position of the robot 
        self.btax = self.fig.add_axes([0.8, 0.02, 0.2, 0.05])
        
        # Initialising all the variables for holding all the required data
        self.x_data, self.y_data = [], []
        self.pose_data = 'X: 0.0\n\nY: 0.0\n\nOrientation: 0.0'
        self.vel_data = 'Linear Velocity: 0.0\n\nAngular Velocity: 0.0'
        self.goal_data = 'TARGET:\n\n    x: 0.0\n    y: 0.0\n\nDistance to Target: 0.0'
        
        # Defining all the text position and properties
        self.info_ax.text(0.1, 0.92, 'POSITION', style='oblique', fontsize=16,
        bbox={'facecolor': 'red','alpha': 0.5, 'pad': 10})
        self.pose_text = self.info_ax.text(0.05, 0.7, self.pose_data, style='normal', fontsize=10,
        bbox={'facecolor': 'red','alpha': 0,'pad': 0})
        self.info_ax.text(0.1, 0.60, 'VELOCITIES', style='oblique', fontsize=16,
        bbox={'facecolor': 'red','alpha': 0.5, 'pad': 10})
        self.info_ax.text(0.1, 0.25, 'GOAL INFO', style='oblique', fontsize=16,
        bbox={'facecolor': 'red','alpha': 0.5, 'pad': 10})
        self.vel_text = self.info_ax.text(0.05, 0.45, self.vel_data, style='normal', fontsize=10,
        bbox={'facecolor': 'red','alpha': 0, 'pad': 0})
        self.goal_text = self.info_ax.text(0.05, 0.02, self.goal_data, style='normal', fontsize=10,
        bbox={'facecolor': 'red','alpha': 0,'pad': 0})
        
        
    # function for setting the initial state of the axes   
    def plot_init(self):
        """Used for the setting the initial properties of the plot to be animated. 

        Returns:
            [matplotlib.artist.Artist]: it returns the artist objects (line, text, circle....). 
            This is done so as to know the current state of the artist in order to update the changes to 
            the artist as oppose recreating the whole artist. 
        """
        self.ax.set_xlim(-7.5, 7.5)
        self.ax.set_ylim(-7.5, 7.5)
        return self.line, self.pose_text, self.vel_text, self.goal_text
    
    # callback function for taking the Odometry message from the odom topic
    def odom_callback(self, msg):
        """This is a callback function for handling the data coming from the odom topic 

        Args:
            msg (Odometry): Odomtetry message coming from the odom topic 
        """
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        self.l_vel = msg.twist.twist.linear.x
        self.a_vel = msg.twist.twist.angular.z
        rotation = msg.pose.pose.orientation
        quaternion = [rotation.x, rotation.y, rotation.z, rotation.w]
        # Using tuple unpacking to get the roll, pitch and yaw values for the euler tuple
        (roll, pitch, self.yaw) = euler_from_quaternion(quaternion)
      
    # function that returns the distance of the robot from the target 
    def the_distance_to_target(self, target):
        """This function is used for calculating the distance of the robot fram the goal 
        position. 

        Args:
            target: The target position of the goal point. 

        Returns:
            [float]: The value of the distance between the robot and the target pose. 
        """

        dist_x = target[0] - self.x
        dist_y = target[1] - self.y

        distance_to_target = sqrt((dist_x * dist_x) + (dist_y * dist_y))
        return distance_to_target
            
    # An update function that would be repeatedly called by the FuncAnimation function
    # this function helps update the state of the figure
    def update_plot(self, frame): 
        """The is the function called every sec to update the Pose Visualization figure. 

        Args:
            frame (int): The number of frames that has reached. 

        Returns:
            [matplotlib.artist.Artist]: it returns the artist objects (line, text, circle....). 
            This is done so as to know the current state of the artist in order to update the changes to 
            the artist as oppose recreating the whole artist. 
        """

        # for updating the line artist based on the most resent data for plotting the line. 
        self.line.set_data(self.x_data, self.y_data)
        

        if len(self.x_data) != 0:

            # for updating the arrow for orientation and the circle for robot pose.
            self.arrow.remove()
            self.circle.remove()
            self.x = self.x_data[-1]
            self.y = self.y_data[-1]
            self.arrow = Arrow(self.x, self.y, (cos(self.yaw)), (sin(self.yaw)))
            self.ax.add_patch(self.arrow)
            self.circle = Circle((self.x, self.y), radius=0.4, label='Robot Pose')
            self.ax.add_patch(self.circle)
            
            # for updating the value of the data displayed on the information axes. 
            self.pose_data = f'X: {self.x:.4f}\n\nY: {self.y:.4f}\n\nOrientation: {self.yaw:.4f}'
            self.vel_data = f'Linear Velocity: {self.l_vel:.4f}\n\nAngular Velocity: {self.a_vel:.4f}'
            if rospy.has_param('/target_point'):
                target_point = rospy.get_param('/target_point')
                distance_to_target = self.the_distance_to_target(target_point)
                self.goal_data = f'TARGET:\n\n    x: {target_point[0]:.4f}\n    y: {target_point[1]:.4f}\n\nDistance to Target: {distance_to_target:.4f}'
            self.pose_text.set_text(self.pose_data)
            self.vel_text.set_text(self.vel_data)
            self.goal_text.set_text(self.goal_data)
            
        if rospy.has_param('/target_point'):
            target_point = rospy.get_param('/target_point')
            self.target.remove()
            self.target = Circle((target_point[0], target_point[1]), radius=0.2, color='r', label='Goal Point')
            self.ax.add_patch(self.target)
        #self.line.set_data(self.x_data, self.y_data)  # to be confirmed*****
        self.ax.legend(loc=1, shadow=True, facecolor='#faebd7', fontsize=8)
        return self.line, self.pose_text, self.vel_text, self.goal_text
    
    def clear_path(self, event):
        """This is the callback function for handling the clear path event,
        when the function is being called, it sets the x and y axis to an empty list
        """
        self.x_data = []
        self.y_data = []
        

In [3]:

class AnalysisPlots:
    """This is a class for the creation of the figure containing an axis for plotting a bar plot for 
    reached and canceled targets and an axes containing an axes for plot the histogram for visualizing the
    frequency of the time is takes the robot to reach a target. 
    """
    def __init__(self):
        self.reached_target = 0
        self.canceled_target = 0
        self.time_to_target = []
        self.fig = plt.figure(figsize=(8,8))
    
    def get_analysis_data(self):
        """This function is used for getting the value of all the required data that is being stored in a
        parameter server by the go to point ros node. 
        """
        if (rospy.has_param('/reached_target')):
            self.reached_target = rospy.get_param('/reached_target')
        if (rospy.has_param('/canceled_target')):
            self.canceled_target = rospy.get_param('/canceled_target')
        if (rospy.has_param('/target_time')):
            self.time_to_target = rospy.get_param('/target_time')
            
    def init_barplot(self):
        """This is function used for defining the initial properties of the barplot. 
        """
        self.axes_2 = self.fig.add_axes([0.1, 0.1, 0.7, 0.1])
        self.axbar = self.fig.add_axes([0.7, 0, 0.15, 0.05])
        self.axes_2.set_title('Number of Canceled and Reached Targets')
        self.get_analysis_data()
        self.axes_2.barh(('Reached\nTarget', 'Canceled\nTarget'),(self.reached_target, self.canceled_target),color=[ 'green','red'])
              
    def init_histplot(self):
        """This is a function used for defining the initial properties of the histplot. 
        """
        self.axes_1 = self.fig.add_axes([0.1, 0.4, 0.7, 0.5])
        self.axhist = self.fig.add_axes([0.7, 0.3, 0.15, 0.05])
        self.axes_1.set_title('Time to Reach Targets')
        self.axes_1.set_xlabel('Time to reach target')
        self.axes_1.set_ylabel('Frequency')
        self.get_analysis_data()
        self.axes_1.hist(self.time_to_target, stacked=True, color='skyblue')

    def reload_bar(self, event):
        """This is a callback function used for handling the reload bar plot event. 
        it updates the plot with the most recent data gotten from the parameter server. 
        """
        self.axes_2.cla()
        self.init_barplot()
    
    
    def reload_hist(self, event):
        """This is a callback function used for handling the reload hist plot event. 
        it updates the plot with the most recent data gotten from the parameter server. 
        """
        self.axes_1.cla()
        self.init_histplot()


In [4]:
        
class VelocityAnalysis:
    """This is a class used for the creation of the figure that contains the axes for displaying 
    the actual and cmd_vel velocities of the robot, both (linear and angular)
    """
    def __init__(self):
        self.fig = plt.figure(figsize=(8, 6))

        # Creating the axis to plot on
        self.ax1 = self.fig.add_axes([0.1, 0.6, 0.8, 0.35])
        self.ax2 = self.fig.add_axes([0.1, 0.1, 0.8, 0.35])

        # Creating the line to add to the axis
        self.line1 = Line2D([], [],color='#c30000', label='Actual Velocity')
        self.line2 = Line2D([], [],color='#2afff9', label='Actual Velocity')
        self.line3 = Line2D([], [],color='#11D005', label='CMD_VEL')
        self.line4 = Line2D([], [],color='#344eff', label='CMD_VEL')

        # Add the lines to the axis 
        self.ax1.add_line(self.line1)
        self.ax2.add_line(self.line2)
        self.ax1.add_line(self.line3)
        self.ax2.add_line(self.line4)

        # Initialize data list
        self.vel1, self.vel11 = [], 0
        self.vel2, self.vel21 = [], 0            
        self.vel3, self.vel31 = [], 0            
        self.vel4, self.vel41 = [], 0
        self.count, self.count1 = [], 0


    def plot_init(self):
        """This function is used for setting the initial properties of the plots (linear, angular)

        Returns:
            [matplotlib.lines.Line2D]: it returns the line artist object. This is done so as to know
            the current state of the artist in order to update the changes to 
            the artist as oppose recreating the whole artist. 
        """
        self.ax1.set_ylim(-2.0, 2.0)
        self.ax1.set_xlim(0, 60)
        self.ax1.set_title('Linear Velocity of Robot in Time')
        self.ax1.set_ylabel('Velocity')
        self.ax1.set_xlabel('Time in Seconds(s)')
        self.ax1.legend(loc=2, shadow=True, facecolor='#faebd7', fontsize=9)
        self.ax2.set_ylim(-2.0, 2.0)
        self.ax2.set_xlim(0, 60)
        self.ax2.set_title('Angular Velocity of Robot in Time')
        self.ax2.set_ylabel('Velocity')
        self.ax2.set_xlabel('Time in Seconds(s)')
        self.ax2.legend(loc=2, shadow=True, facecolor='#faebd7', fontsize=9)
        return self.line1, self.line2, self.line3, self.line4

    def odom_callback(self, msg):
        """This is a callback function for handling the data coming from the odom topic 

        Args:
            msg (Odometry): Odomtetry message coming from the odom topic 
        """
        self.vel11 = msg.twist.twist.linear.x
        self.vel21 = msg.twist.twist.angular.z

    def cmd_callback(self, msg):
        """This is a callback funtion for handling the data coming from the cmd topic

        Args:
            msg (Twist): Twist message coming from the cmd_vel topic 
        """
        self.vel31 = msg.linear.x
        self.vel41 = msg.angular.z

    def update_plot(self, frame):
        """This is a function for updating the plots on each of the axes

        Args:
            frame (int): This is the amount of frames the animation has published.

        Returns:
            [matplotlib.lines.Line2D]: it returns the line artist object. This is done so as to know
            the current state of the artist in order to update the changes to 
            the artist as oppose recreating the whole artist. 
        """
        self.vel1.append(self.vel11)
        self.vel2.append(self.vel21)
        self.vel3.append(self.vel31)
        self.vel4.append(self.vel41)
        self.count.append(self.count1)
        self.count1 += 1
        if (len(self.count) > 60):
            # This for updating the limits of the axes to accomodate the new size of the plot list value
            # also the list size is cut to the size of data that would be displayed on the plot. 
            time_len = self.count1 #len(self.count)
            x_lim = time_len - 60
            self.ax1.set_xlim(x_lim, (60 + x_lim))
            self.ax2.set_xlim(x_lim, (60 + x_lim))
            self.vel1 = self.vel1[-60:]
            self.vel2 = self.vel2[-60:]
            self.vel3 = self.vel3[-60:]
            self.vel4 = self.vel4[-60:]
            self.count = self.count[-60:]
            
        self.line1.set_data(self.count, self.vel1)
        self.line2.set_data(self.count, self.vel2)
        self.line3.set_data(self.count, self.vel3)
        self.line4.set_data(self.count, self.vel4)
        return self.line1, self.line2, self.line3, self.line4


### Required Function

The functions below are the functions used to define the sequence of events that occurs when a user interact with each of the IpyButtons and Sliders

In [5]:
def start_simulation(a):
    ui_client("start")
    start_button.button_style = 'success'
    stop_button.button_style = ''
    
def stop_simulation(b):
    ui_client("stop")
    start_button.button_style = ''
    stop_button.button_style = 'danger'  
    
def move_forward(a):
    if (rospy.has_param('/linear_velocity')):
        lv_multiplier = rospy.get_param('/linear_velocity')
    else:
        lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*0.5
    pub.publish(vel)
    up_button.button_style = 'info'
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = ''
    
    
def move_backward(a):
    if (rospy.has_param('/linear_velocity')):
        lv_multiplier = rospy.get_param('/linear_velocity')
    else:
        lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*(-0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = 'info'
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = ''
  

def turn_left(a):
    if (rospy.has_param('/angular_velocity')):
        av_multiplier = rospy.get_param('/angular_velocity')
    else:
        av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(-0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = 'info'
    right_button.button_style = ''
    stop.button_style = ''
   

def turn_right(a):
    if (rospy.has_param('/angular_velocity')):
        av_multiplier = rospy.get_param('/angular_velocity')
    else:
        av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = 'info'
    stop.button_style = ''

    
def stop_movement(a):
    vel = Twist()
    vel.linear.x = 0.0
    vel.angular.z = 0.0
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = 'danger'
    
    
def on_value_change(change):
    x = change['new']
    rospy.set_param('/linear_velocity', x)
    
    
def on_valueang_change(change):
    z = change['new']
    rospy.set_param('/angular_velocity', z)


def button_event():
    start_button.on_click(start_simulation)
    stop_button.on_click(stop_simulation)
    up_button.on_click(move_forward)
    down_button.on_click(move_backward)
    left_button.on_click(turn_left)
    right_button.on_click(turn_right)
    stop.on_click(stop_movement)
    l_velSlider.observe(on_value_change, names='value')
    a_velSlider.observe(on_valueang_change, names='value')
    

   

In [6]:
## Main function for initialising the ros node and calling all the button events
def main():
    rospy.init_node('user_interface')
    button_event()

### Figure 1
Figure 1 gives both a textual and graphical visualisation of the Robot State and the goal position, it also features a button called **Clear Path**; this button is used for clearing the path that has been plotted. Below the figure is a slider for controlling the rate of the linear and angular velocity of the robot.

In [7]:
%matplotlib widget
rospy.wait_for_service('/user_interface')
try:
    ui_client = rospy.ServiceProxy('/user_interface', Command)
except rospy.ServiceException as e:
    print(f'Service call failed: {e}')
    
main()
vis = Visualiser()
    
pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit=True)

clear_path = Btn(vis.btax, 'Clear Path')
clear_path.on_clicked(vis.clear_path)
plt.show(block=True)
    
VBox([l_velSlider, a_velSlider])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Control Buttons
**Start Button**: The start button is for starting the robot motion, allowng it to go to randomly generated goal position in the 3D Simulation. The button is green when the simulation has started and gray when it is stopped. 

**Stop Buttin**: The stop button is for stoping the simulation. The button is red when the simulation has been stopped and gray when the simulation has started. 

Another set of button is what i call the **"Control Pad"**, this set of buttons consisting of the up, down, left and right arrow can be used to manually control the robot to move around the 3D Simulation environment. Note, it doesnt work when the random goal simulation has been started with the start button; to use these Control Pad, the simulation has to be stopped with the stop button. The Control Pad controls can however be preempted by the Start button to start the random goal simulation 

In [8]:
HBox([start_button, stop_button])

In [9]:
display(control_pad)

GridBox(children=(Button(icon='arrow-up', layout=Layout(grid_area='up', height='50px', width='auto'), style=Bu…

### Figure 2
This figure is used for displaying the *linear* and *angular* velocity of the robot, by comparing the velocity being sent to the robot through the **cmd_vel** and the **actual velocity** that the robot is moving at. 


In [10]:
vel_analysis = VelocityAnalysis()
    
sub2 = rospy.Subscriber('/cmd_vel', Twist, vel_analysis.cmd_callback)
sub3 = rospy.Subscriber('/odom', Odometry, vel_analysis.odom_callback)
anim = FuncAnimation(vel_analysis.fig, vel_analysis.update_plot, init_func=vel_analysis.plot_init, blit=True, interval=1000)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Figure 3
The figure 3 contains two plot the **Histogram plot** and the **Bar plot**. 

**Histogram Plot**: *The histogram plot is used for showing the frequency of time it take the robot to reach the goal position.*

**Bar Plot**: _The Bar Plot is used for visualising the number of **Reached Targets** and **Canceled Target**_

These plot are not live plots, to check the most recent plot, two buttons have been provided, **Reload HisPlot** and **Reload BarPlot**; these two button are used to update the plot with the most recent data gotten from the Parameter Server in which they are being stored. 

In [11]:
anaplot = AnalysisPlots()
anaplot.init_barplot()
anaplot.init_histplot()

bplot = Btn(anaplot.axbar, 'Reload BarPlot')
hplot = Btn(anaplot.axhist, 'Reload HistPlot')
bplot.on_clicked(anaplot.reload_bar)
hplot.on_clicked(anaplot.reload_hist)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …